In [ ]:
import torch
import torch.nn as nn

Dataset: :32×32 RGB images, 10 classes


    CNN with one convolutional layer and ReLU activation.
    Architecture:
    -Conv2d: 3 → 16 channels,kernel=3×3,stride=1,padding=1
    -activation=ReLU



In [ ]:

class BasicConvNet(nn.Module):

    def __init__(self, in_channels=3, out_channels=16):
        super(BasicConvNet, self).__init__()

        # Convolutional layer
        self.conv1 = nn.Conv2d(
            in_channels=in_channels,
            out_channels=out_channels,
            kernel_size=3,
            stride=1,
            padding=1
        )

         # ReLU activation
        self.relu = nn.ReLU()

    def forward(self, x):
        """
        Args:
            x: Input shape (batch_size, 3, 32, 32)
        """
        x = self.conv1(x)
        x = self.relu(x)
        return x

# CALCULATION - STEP 1:

# Formula for Conv2d parameters:
#  Parameters = (kernel_height × kernel_width × in_channels

# Given:
# in_channels = 3 (RGB)
# out_channels = 16
# kernel_size = 3×3

# Calculation:
# Weights = 3 × 3 × 3 × 16 = 432
# Biases = 16
# Total = 432 + 16 = 448 parameters

# Output shape calculation:
#H_out = (H_in + 2×padding - kernel_size) / stride + 1
#H_out = (32 + 2×1 - 3) / 1 + 1 = 32
#W_out = (32 + 2×1 - 3) / 1 + 1 = 32



    Extended CNN with convolutional block + FC classifier.
    
    Architecture:
    - Conv2d: 3 → 16 channels, kernel=3×3, stride=1, padding=1
    -activation=ReLU
    -Flatten
    -Linear: 16×32×32 → 10 (output classes)


In [ ]:
# STEP 2: Extended Network with Fully Connected Layer

class ExtendedConvNet(nn.Module):

    def __init__(self, in_channels=3, out_channels=16, num_classes=10):
        super(ExtendedConvNet, self).__init__()

        # Convolutional layer
        self.conv1 = nn.Conv2d(
            in_channels=in_channels,
            out_channels=out_channels,
            kernel_size=3,
            stride=1,
            padding=1
        )

        # ReLU activation
        self.relu = nn.ReLU()

        # Fully connected layer
        # Input: 16 channels × 32 height × 32 width = 16,384 features
        self.fc = nn.Linear(
            in_features=out_channels * 32 * 32,
            out_features=num_classes
        )

    def forward(self, x):


        x = self.conv1(x)
        x = self.relu(x)

        x = x.view(x.size(0), -1)

        # Fully connected layer
        x = self.fc(x)

        return x


# PARAMETER CALCULATION - STEP 2:

# 1. Convolutional Layer (from Step 1):
#    - Parameters = 448

# 2. Fully Connected Layer:
#    Given:
#    -in_features = 16 × 32 × 32 = 16,384
#    -out_features = 10

#    Calculation:
#    -Weights = 16,384 × 10 = 163,840
#    -Biases = 10
#    -Total FC = 163,840 + 10 = 163,850 parameters

# 3. TOTAL NETWORK PARAMETERS:
#    -Conv layer: 448
#    -FC layer: 163,850
#    -GRAND TOTAL: 164,298 parameters


In [ ]:
# TEST FUNCTIONS
def test_basic_conv():


    net = BasicConvNet(in_channels=3, out_channels=16)


    x = torch.randn(1, 3, 32, 32)


    y = net(x)


    assert y.shape == (1, 16, 32, 32), f"Expected shape (1, 16, 32, 32), got {y.shape}"

    total_params = sum(p.numel() for p in net.parameters())

    print(f"Input shape: {tuple(x.shape)}")
    print(f"Output shape: {tuple(y.shape)}")
    print(f"Total parameters: {total_params}")
    print(f"Expected parameters: 448")

    return net



In [ ]:
def test_extended_conv():

    net = ExtendedConvNet(in_channels=3, out_channels=16, num_classes=10)


    x = torch.randn(1, 3, 32, 32)


    y = net(x)


    assert y.shape == (1, 10), f"Expected shape (1, 10), got {y.shape}"


    total_params = sum(p.numel() for p in net.parameters())

    print(f"Input shape: {tuple(x.shape)}")
    print(f"Output shape: {tuple(y.shape)}")
    print(f"Total parameters: {total_params}")
    print(f"parameters: 164,298")

    # Detailed parameter breakdown
    print("Parameter Breakdown:")
    print("-" * 60)
    for name, param in net.named_parameters():
        print(f"{name:20s}: {param.numel():>10,} parameters, shape {tuple(param.shape)}")
    print("-" * 60)
    print(f"{'TOTAL':20s}: {total_params:>10,} parameters\n")

    return net


In [ ]:
def test_batch_processing():
    net = ExtendedConvNet()

    batch_size = 8
    x = torch.randn(batch_size, 3, 32, 32)
    y = net(x)

    assert y.shape == (batch_size, 10), f"Expected shape ({batch_size}, 10), got {y.shape}"

    print(f"Batch input shape: {tuple(x.shape)}")
    print(f"Batch output shape: {tuple(y.shape)}")

In [ ]:
# MAIN EXECUTION
if __name__ == "__main__":
    # Run all tests
    test_basic_conv()
    test_extended_conv()
    test_batch_processing()

Input shape: (1, 3, 32, 32)
Output shape: (1, 16, 32, 32)
Total parameters: 448
Expected parameters: 448
Input shape: (1, 3, 32, 32)
Output shape: (1, 10)
Total parameters: 164298
parameters: 164,298
Parameter Breakdown:
------------------------------------------------------------
conv1.weight        :        432 parameters, shape (16, 3, 3, 3)
conv1.bias          :         16 parameters, shape (16,)
fc.weight           :    163,840 parameters, shape (10, 16384)
fc.bias             :         10 parameters, shape (10,)
------------------------------------------------------------
TOTAL               :    164,298 parameters

Batch input shape: (8, 3, 32, 32)
Batch output shape: (8, 10)
